<a href="https://colab.research.google.com/github/ttogle918/news_by_kobert/blob/master/MLM/BertForMaskedLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 패키지 불러오기

In [ ]:
!pip install sentencepiece transformers torch

In [ ]:
!pip install seqeval
!pip install torchtext pytorch-lightning
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
!pip install sentence_transformers numpy scikit-learn scipy nltk tqdm

In [1]:
import pytorch_lightning as pl
pl.__version__

'1.5.10'

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [3]:
from transformers import BertConfig, AdamW
from transformers import BertForMaskedLM
from transformers import FillMaskPipeline
from transformers import BertTokenizer, AutoTokenizer

In [101]:
tokenizer = BertTokenizer.from_pretrained(config.bert_model)
model = BertForMaskedLM.from_pretrained(config.bert_model)

Some weights of the model checkpoint at snunlp/KR-Medium were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [102]:
inputs = tokenizer("축구는 정말 재미있는 [MASK]다.", return_tensors="pt")
labels = tokenizer("축구는 정말 재미있는 스포츠다.", return_tensors="pt")["input_ids"]

In [103]:
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

In [110]:
outputs

MaskedLMOutput([('loss', tensor(3.2178, grad_fn=<NllLossBackward0>)),
                ('logits',
                 tensor([[[-5.4640,  1.0087, -3.7623,  ..., -4.1494, -4.8447, -5.1425],
                          [-4.6789,  1.8529, -4.5086,  ..., -2.8888, -3.3678, -2.8439],
                          [-7.7065,  1.0267, -6.0396,  ..., -5.3454, -8.1307, -5.2234],
                          ...,
                          [-6.4272, -0.8331, -6.3297,  ..., -4.2100, -4.5536, -6.1272],
                          [-6.9698,  7.7743,  1.2039,  ..., -6.5364, -7.8356, -5.0088],
                          [-5.1348,  0.8143, -3.9422,  ..., -3.2602, -5.2708, -3.5649]]],
                        grad_fn=<AddBackward0>))])

In [109]:
model(input_ids = inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)

MaskedLMOutput([('loss', tensor(3.2178, grad_fn=<NllLossBackward0>)),
                ('logits',
                 tensor([[[-5.4640,  1.0087, -3.7623,  ..., -4.1494, -4.8447, -5.1425],
                          [-4.6789,  1.8529, -4.5086,  ..., -2.8888, -3.3678, -2.8439],
                          [-7.7065,  1.0267, -6.0396,  ..., -5.3454, -8.1307, -5.2234],
                          ...,
                          [-6.4272, -0.8331, -6.3297,  ..., -4.2100, -4.5536, -6.1272],
                          [-6.9698,  7.7743,  1.2039,  ..., -6.5364, -7.8356, -5.0088],
                          [-5.1348,  0.8143, -3.9422,  ..., -3.2602, -5.2708, -3.5649]]],
                        grad_fn=<AddBackward0>))])

In [54]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)
pip('축구는 정말 재미있는 [MASK]다.')

[{'score': 0.3231074810028076,
  'sequence': '축구는 정말 재미있는 스포츠 다.',
  'token': 12302,
  'token_str': '스 포 츠'},
 {'score': 0.2157781571149826,
  'sequence': '축구는 정말 재미있는 경기 다.',
  'token': 9195,
  'token_str': '경 기'},
 {'score': 0.1995808333158493,
  'sequence': '축구는 정말 재미있는 대회 다.',
  'token': 11909,
  'token_str': '대 회'},
 {'score': 0.04826343059539795,
  'sequence': '축구는 정말 재미있는 축구 다.',
  'token': 11062,
  'token_str': '축 구'},
 {'score': 0.019712088629603386,
  'sequence': "축구는 정말 재미있는'다.",
  'token': 11,
  'token_str': "'"}]

In [58]:
pip(tokenizer.decode(torch.tensor([2,5,6,7,4,3])))

[{'score': 0.45599833130836487,
  'sequence': '! " # "',
  'token': 6,
  'token_str': '"'},
 {'score': 0.040783338248729706,
  'sequence': '! " #!',
  'token': 5,
  'token_str': '!'},
 {'score': 0.021625056862831116,
  'sequence': '! " # 1',
  'token': 21,
  'token_str': '1'},
 {'score': 0.019090600311756134,
  'sequence': '! " #',
  'token': 1,
  'token_str': '[ U N K ]'},
 {'score': 0.018774133175611496,
  'sequence': '! " # 5',
  'token': 25,
  'token_str': '5'}]

In [5]:
from tqdm import tqdm, tqdm_notebook
from typing import Callable, List, Tuple
from seqeval.metrics import accuracy_score

In [6]:
import torch
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device, torch.cuda.device_count()

(device(type='cuda', index=0), 1)

In [7]:
import random
# text가 7문장(다.로 끝나는 것)이라면 5 * 4 이하의 data가 나온다.(중복일 때 data가 줄어든다.)
def text_splicing(text) :
  text = text.replace('\n', ' ').split('다.')
  if text[-1].strip() == '':
    text = text[:-1]
  if len(text) < 2 :
    return None
  
  before_text = text[0] + '다. '
  content = []
  for i, t in enumerate(text[1:-1]) :
    t += '다. '
    if len(content) >= 3 :
      for j in range(-len(selected), 0) :
        content[j][0] += t    # 다음문장 연결
        content[j][0] = content[j][0].replace('  ', ' ').strip()
        content[j][1] += t    # 다음문장 연결
        content[j][1] = content[j][1].replace('  ', ' ').strip()
    
    div_space_t = t.strip().split()
    if len(div_space_t) <= 4 :   # 문장이 4 단어 이하로 이루어져있다면 이전 문장에 덧붙이고 다음문장으로
      before_text += t
      continue

    selected = []
    
    for i in range(3) :   # [MASK] 3문장
      rd = random.randint(0, len(div_space_t)-2)
      selected.append(rd)

    selected = set(selected)    # 중복 제거
    selected = list(selected)
    
    rd = random.randint(0, len(div_space_t)-3)    # [MASK] [MASK] ( 띄어쓰기로 단락 분리된 경우, 2단어) 1문장
    selected.append(rd)   # 마지막 idx는 mask 두개
    
    for s in selected[:-1] :
      label = before_text + t
      sentence = before_text + ' '.join(div_space_t[:s]) + " [MASK] " + ' '.join(div_space_t[s+1:])
      content.append([sentence, label])
    
    content.append([before_text + ' '.join(div_space_t[:s]) + " [MASK] [MASK] " + ' '.join(div_space_t[s+2:]), before_text+t])
    # content[-1][1] = [div_space_t[rd], div_space_t[rd+1]]
    before_text = t

  for j in range(-len(selected), 0) :
      content[j][0] += text[-1] + '다.'    # 다음문장 연결
      content[j][1] += text[-1] + '다.'    # 다음문장 연결
      content[j][0] = content[j][0].replace('  ', ' ').strip()
      content[j][1] = content[j][1].replace('  ', ' ').strip()

  return content

In [8]:
from google.colab import drive

drive.mount('/content/drive')
data_path = '/content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400'

Mounted at /content/drive


In [ ]:
import os
dataset = []
for (path, dir, files) in os.walk(data_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            with open("%s/%s" % (path, filename), encoding="utf-8") as f:
              label = path[path.rindex('/')+1:]
              text = f.read()
              temp_list = text_splicing(text)
              if temp_list is not None :
                dataset.extend(temp_list)

len(dataset), len(dataset[0])

In [10]:
len(dataset), len(dataset[0])

(457443, 2)

In [11]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train, test = train_test_split(dataset[:1000], test_size=0.2, random_state=0, shuffle=True)
train, val = train_test_split(train, test_size=0.2, random_state=0, shuffle=True)
print("train :", len(train), "  val :", len(val), "  test :", len(test))

train : 640   val : 160   test : 200


In [12]:
train[-1]

['왜 자신의 눈 속의 대들보는 보지 못하고 국민의 눈 속에 있는 티만 보려 하는가 말이다. 왜 검찰은 스스로의 잘못에 의해 자신들의 훌륭한 선후배들이 옷을 벗게 되었는데도 그 이유가 어디에 있는지 아직도 모른 채 [MASK] 낙마를 남의 탓으로 돌리고 있는지 알 수가 없다. 이명박 대통령도 말하지 않았는가? 다른 것은 몰라도 거짓말을 하는 것은 용납할 수 없다고 말이다.',
 '왜 자신의 눈 속의 대들보는 보지 못하고 국민의 눈 속에 있는 티만 보려 하는가 말이다. 왜 검찰은 스스로의 잘못에 의해 자신들의 훌륭한 선후배들이 옷을 벗게 되었는데도 그 이유가 어디에 있는지 아직도 모른 채 천성관의 낙마를 남의 탓으로 돌리고 있는지 알 수가 없다. 이명박 대통령도 말하지 않았는가? 다른 것은 몰라도 거짓말을 하는 것은 용납할 수 없다고 말이다.']

In [13]:
del dataset
del text

## CorpusDataset

In [14]:
class CorpusDataset(Dataset):
    def __init__(self, sentences, transform: Callable):
        self.sentences = sentences
        self.transform = transform

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx][0]
        label = self.sentences[idx][1]
        (
            input_ids,
            attention_mask,
            token_type_ids,
            label, 
        ) = self.transform(sentence, label)

        return input_ids, attention_mask, token_type_ids, label

## Preprocessor

In [15]:
from typing import List, Tuple

class Preprocessor :
    def __init__(self, max_len: int):
        self.tokenizer = BertTokenizer.from_pretrained("snunlp/KR-Medium", do_lower_case=False)
        self.max_len = max_len
        self.pad_token_id = 0

    def get_input_features(self, sentence, labels
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """문장에 대해 tokenize한 뒤 id로 변환, token_type_ids는 의미없다.

        Args:
            sentence: mask가 포함된 문장
            label : mask의 값

        Returns:
            feature를 리턴한다.
            input_ids : mask가 있는 문장의 encode, 2 ..4... 3
            attention_mask : padding은 0, 데이터가 존재하면 1
            token_type_ids : 0... ( 한 문장 )
            label : label 문장의 encode, 2, ... , 3
        """
        input_ids = self.tokenizer.encode(sentence)
        label = self.tokenizer.encode(labels)

        find_mask_idx = input_ids.index(4)

        if len(input_ids) < len(label) :
          input_ids = input_ids[:find_mask_idx] + [4] * (len(label)-len(input_ids)) + input_ids[find_mask_idx:]    # input_ids가 다를 때 mask 추가 

        if len(input_ids) > self.max_len :  # max_len = 128
          # mask 위치를 찾아서 먼 곳부터 자르기? mask : 4
          start = find_mask_idx - self.max_len//2
          start = 0 if start < 0 else start
          input_ids = input_ids[start:start+self.max_len]
          label = label[start:start+self.max_len]


        attention_mask = [1] * len(label)

        pad_length = self.max_len-len(label)

        input_ids.extend([0] * pad_length)
        label.extend([0] * pad_length)
        # token_type_ids.extend([0] * pad_length) # pad : 0

        token_type_ids = [0]*self.max_len

        # token_type_ids[find_mask_idx] = 1       # mask가 있는 idx에는 token_type_ids가 1이다. ( 문장 분리)
        attention_mask.extend([0] * pad_length) # pad : 0

        input_ids = torch.tensor(input_ids, dtype=torch.int)
        attention_mask = torch.tensor(attention_mask, dtype=torch.int)
        token_type_ids = torch.tensor(token_type_ids, dtype=torch.int)
        label = torch.tensor(label, dtype=torch.int)

        return input_ids, attention_mask, token_type_ids, label

## BERTNextSentenceModel

In [132]:
from math import log
class BertOnlyNSPHead(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.seq_relationship = nn.Linear(config.hidden_size, 2)

    def forward(self, pooled_output):
        seq_relationship_score = self.seq_relationship(pooled_output)
        print(seq_relationship_score, len(seq_relationship_score))
        return seq_relationship_score

class mBertForMaskedLM(pl.LightningModule):
    def __init__(self, config, dataset,  num_labels=2):
        super().__init__()
        self.config = config
        self.dataset = dataset
        self.pad_token_id = 0
        self.softmax = torch.nn.Softmax()
        self.bert_config = BertConfig.from_pretrained(
            self.config.bert_model
        )
        self.model = BertForMaskedLM.from_pretrained(
            self.config.bert_model, config=self.bert_config
        ).cuda()
        # self.tokenizer = AutoTokenizer.from_pretrained(self.config.bert_model, do_lower_case=False)
        self.cls = BertOnlyNSPHead(config)
        self.dropout = nn.Dropout(self.config.dropout_rate)
        self.linear = nn.Linear(
            self.bert_config.hidden_size, num_labels
        )
        # self.pipeline = FillMaskPipeline(self.model, self.tokenizer).cuda()

    def forward(self,
        input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None,
        head_mask=None, inputs_embeds=None, labels=None, output_attentions=None, output_hidden_states=None,
        return_dict=None, **kwargs, ):
        """
          self.model의 결과물 :  
          
          NextSentencePredictorOutput(
            loss=next_sentence_loss,
            logits=seq_relationship_scores,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
          )

        """
        print("forward")
        print(labels)
        output = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        # probs = self.softmax(output.logits)
        print(output)
        return output

    def training_step(self, batch, batch_nb):
        print("training_step")
        input_ids, attention_mask, token_type_ids, label_ids = batch
        loss = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=label_ids,
        )
        print("loss", loss)
        # acc = self._calculate_accuracy(input_ids, label_ids)
        loss = self._calculate_loss(loss)

        tensorboard_logs = {'train_loss': loss}
        return {"loss": loss, "log": tensorboard_logs}

    def training_end(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, label_ids = batch

        loss, logits = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        acc = self._calculate_accuracy(input_ids, label_ids)
        loss = self._calculate_loss(loss)

        tensorboard_logs = {'train_loss': loss}
        print("training_end : ", tensorboard_logs)
        return {"loss": loss, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        super().on_train_epoch_end()
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        acc = torch.stack([x['acc'] for x in outputs]).mean()
        tensorboard_logs = {'loss': avg_loss, 'acc': acc}
        print('training_epoch_end : ', tensorboard_logs)
        self.log(tensorboard_logs)
        self.log("training_epoch_end : tensorboard_logs ", tensorboard_logs)

    def validation_step(self, batch, batch_nb):
        print("validation_step : ")
      
        input_ids, attention_mask, token_type_ids, label_ids = batch
        output = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels = label_ids
        )
        loss = self._calculate_loss(output)
        print("loss", loss)

        # val_acc = self._calculate_accuracy(input_ids, label_ids)

        return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        print(outputs)
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        # val_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        print('validation_epoch_end : ', tensorboard_logs)
        self.log(tensorboard_logs)
        return {'val_loss': avg_loss, 'log': tensorboard_logs}
    
    def validation_end(self, outputs):
        val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        # val_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": val_loss }
        return {'val_loss': val_loss, 'log': tensorboard_logs}
    
    def test_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, label_ids = batch
        loss = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        # acc = self._calculate_accuracy(input_ids, label_ids)
        loss = self._calculate_loss(loss)

        return {"test_loss": loss}

    def test_end(self, outputs):
        test_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        # test_acc = torch.stack([x["test_acc"] for x in outputs]).mean()
        return {"labels" : [x["labels"] for x in outputs], "test_loss": test_loss}
    
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        # acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss}
        print('test_epoch_end : ', tensorboard_logs)
        self.log(tensorboard_logs)
        return {'test_loss': avg_loss, 'log': tensorboard_logs}

    def predict_step(self, batch, batch_idx, dataloader_idx=0):   # prediction : forward(), predict_step()
        input_ids, attention_mask, token_type_ids, _ = batch    # slot_label은 없음.
        loss = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        return {'pred_labels':loss}

    def configure_optimizers(self):
        return AdamW(self.model.parameters(), lr=2e-5, eps=1e-8)

    def train_dataloader(self):
        return DataLoader(self.dataset["train"], batch_size=self.config.eval_batch_size)

    def val_dataloader(self):
        return DataLoader(self.dataset["val"], batch_size=self.config.eval_batch_size)

    def test_dataloader(self):
        return DataLoader(self.dataset["test"], batch_size=self.config.eval_batch_size)

    def pred_dataloader(self, dataset):
        return DataLoader(dataset, batch_size=1)

    def _calculate_loss(self, outputs):   # 확률에서 얼마나 떨어져있는가?
        # loss = F.cross_entropy(outputs, labels)

        return outputs.loss

    def pipelining(self, input_id) :
        return self.tokenizer.decode(input_id)

    def _calculate_accuracy(self, input_ids, label_ids):   # 0.5보다 크면 1, 아니면 0으로 labeling
        acc_sum = 0
        batch_size = 0
        for input_id, label_id in zip(input_ids, label_ids) :
          batch_size += 1
          result = self.pipeline(self.pipelining(input_id), top_k = 10)

          mask_idx=(input_id == 4).nonzero(as_tuple=True)[0]   # input_id.index(4) tensor버전
          mini_sum = 0
          for i, m in enumerate(mask_idx) :
            if len(mask_idx) == 1 :
              for r in result :   # result[0]이 dict라면
                if r['token'] == label_id[m].int() :
                  acc_sum += 1   # 일부를 맞췄다면 그 비율만큼 정확도 더하기
                  break
            else :
              for r in result[i] :
                # if type(r) != dict :
                #   print(type(r), r, result[i], result)

                #   break
                if r['token'] == label_id[m].int() :
                  mini_sum += 1   # 일부를 맞췄다면 그 비율만큼 정확도 더하기
                  break
              acc_sum += mini_sum / len(mask_idx)
        return acc_sum / batch_size

## Config

In [17]:
# yaml 파일 대신에 객체로 생성
class Config(BertConfig):
  def __init__(self) :
    super().__init__()
    self.task= 'kor_BertForMaskedLM_'
    self.log_path= data_path+'/logs'
    self.bert_model = "snunlp/KR-Medium"
    self.max_len= 128
    self.train_batch_size= 64
    self.eval_batch_size= 64
    self.dropout_rate= 0.1
    self.gpus= torch.cuda.device_count()
config = Config()

## Train

In [18]:
preprocessor = Preprocessor(config.max_len)

Downloading:   0%|          | 0.00/140k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

In [19]:
dataset = {}
dataset['train'] = CorpusDataset(train, preprocessor.get_input_features)
dataset['val'] = CorpusDataset(val, preprocessor.get_input_features)
dataset['test'] = CorpusDataset(test, preprocessor.get_input_features)

In [107]:
dataset['train']

In [21]:
print("train")
for i, d in enumerate(dataset['train']) :
  if len(d[0]) == len(d[1]) == len(d[2]) == len(d[3]) == 128 :
    pass
  else :
    print(i, len(d[0]), len(d[1]), len(d[2]), len(d[3]))
print("val")
for i, d in enumerate(dataset['val']) :
  if len(d[0]) == len(d[1]) == len(d[2]) == len(d[3]) == 128 :
    pass
  else :
    print(i, len(d[0]), len(d[1]), len(d[2]), len(d[3]))

print("test")
for i, d in enumerate(dataset['test']) :
  if len(d[0]) == len(d[1]) == len(d[2]) == len(d[3]) == 128 :
    pass
  else :
    print(i, len(d[0]), len(d[1]), len(d[2]), len(d[3]))

train
val
test


In [22]:
import gc
gc.collect()

54

In [133]:
model = mBertForMaskedLM(config, dataset).cuda()

Some weights of the model checkpoint at snunlp/KR-Medium were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [134]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor

logger = TensorBoardLogger(
    save_dir=os.path.join(config.log_path, config.task), version=1, name=config.task
)

acc_checkpoint_callback = ModelCheckpoint(
    dirpath=data_path+'/MLM/checkpoints/'+ config.task, 
    filename="{epoch}_{val_acc:4f}_{val_loss:.4f}",
    verbose=True,
    monitor='val_acc',
    mode='max',
    save_top_k=1,
    save_last=True)

lrmonitor_callback = LearningRateMonitor(logging_interval='step')

In [135]:
import gc
gc.collect()

3758

In [ ]:
trainer = pl.Trainer(
    gpus=config.gpus,
    callbacks=[acc_checkpoint_callback, lrmonitor_callback],
    logger=logger,
    max_epochs=1,
)

trainer.fit(model)

# RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Int'